### Install Dependensi
Dependensi yang dibutuhkan hanya Numpy yang kita import sebagai alias np

In [ ]:
%pip install numpy 

import numpy as np
'''
numpy yang sudah / akan digunakan:
1. np.linalg.det()
2. np.sqrt()
3. np.dot()
4. np.asin()
5. np.acos()
6. np.array()
7. np.cross()
8. np.dot()
9. np.linalg.norm()
'''

### Helper Function
Beberapa function yang akan kita gunakan beberapa kali

In [ ]:
def hitungDet(matrix): # hitung determinan langsung pakai numpy
    det = np.linalg.det(matrix)
    return det

def invalidInputMessageDeterminan(r, c): # error handling kalo ada input yang tidak sesuai
    print(f"Determinan tidak dapat ditentukan untuk matrix berukuran {r} x {c}")
    print("Hanya dapat menghitung matrix persegi (n x n)")
    print()

def tampilkanMatrix(matrix): # menampilkan matrix yang ingin ditampilkan
    for row in matrix:
        atur = ["{:.2f}".format(x) for x in row]
        print("    [" + ", ".join(atur) + "]")

def tampilkanMatrix1D(v):
    print("[", end = '')
    print(", ".join(f"{x:.2f}" for x in v), end = '')
    print(']')

def displayInput(vectors):
    # tampilkan semua matrix
    for i, v in enumerate(vectors):
        print(f"    V{i+1} =", end=' ')
        tampilkanMatrix1D(v)
        print()


def displayInput2(vectors):
    # tampilkan semua matrix
    for i, v in enumerate(vectors):
        print(f"    E{i+1} =", end=' ')
        tampilkanMatrix1D(v)
        print()

def displayInput3(vectors):
    # tampilkan semua matrix
    for i, v in enumerate(vectors):
        print(f"    P{i+1} =", end=' ')
        tampilkanMatrix1D(v)
        print()

def displayInput4(vectors):
    # tampilkan semua matrix
    for i, v in enumerate(vectors):
        print(f"    r+t[{i+1}] =", end=' ')
        tampilkanMatrix1D(v)
        print()

def displayInput5(vectors):
    # tampilkan semua matrix
    for i, v in enumerate(vectors):
        print(f"    r-t[{i+1}] =", end=' ')
        tampilkanMatrix1D(v)
        print()

def displayInput6(vectors):
    # tampilkan semua matrix
    for i, v in enumerate(vectors):
        print(f"    U{i+1} =", end=' ')
        tampilkanMatrix1D(v)
        print()

def displayInput7(vectors):
    # tampilkan semua matrix
    for i, v in enumerate(vectors):
        print(f"    W{i+1} =")
        tampilkanMatrix(v)
        print()

def displayInput8(vectors):
    # tampilkan semua matrix
    for i, v in enumerate(vectors):
        print(f"    T{i+1} =", end=' ')
        tampilkanMatrix1D(v)
        print()

def radToDeg(res):
    ans = res * (180 / np.pi)
    return ans

def degToRad(res):
    ans = res * (np.pi / 180)
    return ans
    

### Program PPT 1 - Menyelesaikan Sistem Persamaan Linear dengan Algoritma OBE Gauss-Jordan

Pseudocode:

1. Buat matrix input
2. Buat function calls ke function utama
3. Di function utama, buat utility yang diinginkan misal print judul / header, matrix input, dan sebagainya
4. Cek dulu matrixnya kalau banyak baris tidak sama dengan banyak kolom-1 maka tampilkan pesan error (di function helper)
5. Di function utama bisa langsung buat algoritma gauss jordan:
    - Ambil pivot di diagonal yang bukan 0, jika 0 tukar baris dengan baris di bawahnya
    - Normalisasi pivot agar menjadi 1 utama (jika sudah satu utama skip)
    - Lakukan pengurangan untuk baris di bawahnya agar di bawah 1 utama semuanya 0
    - Lakukan terus hingga kolom terakhir sudah tidak ada baris di bawah 1 utama, maka lakukan lagi tetapi untuk baris di atasnya dengan algoritma yang sama
    - Tampilkan hasil matrix setiap iterasi
6. Kondisi tidak ada solusi juga terjadi ketika persamaan tidak konsisten (diagonal mendektai 0 tetapi konstanta tidak 0) dan banyak solusi (diagonal dan konstanta keduanya mendekati 0)
7. Tampilkan output hasil solusi persamaan
done!


In [ ]:
def tampilkanPersamaan(matrix):
    row = len(matrix)
    col = len(matrix[0])
    variabel = col - 1

    for i in range(row):
        left = []
        for j in range(variabel):
            coef = matrix[i][j]
            left.append(f"({coef:.2f})X{j+1}")

        lhs = " + ".join(left)
        rhs = matrix[i][col-1]

        print(f"{lhs} = {rhs:.2f}")


def iterateGaussJordan(matrix):
    swap = 0        # hitung berapa kali tukar baris
    row = len(matrix)
    col = len(matrix[0])
    M = [r[:] for r in matrix]      # copy matrix ke M, aku gk mau modif matrix awal, resiko kehilangan data awal
    step = 1

    for j in range(row): # loop tiap kolom, jadi dia bisa drive gauss (sebagai index baris) sekaligus nentuin pivot. Perhatikan bahwa loop ini berhenti sampai row-1 karena kolom paling kanan tidak di-normalisasi
        pivot = M[j][j]
        if pivot == 0:
            # lakukan tukar baris
            for i in range(j+1, row): # loop tiap baris, cari pivot != 0 di bagian bawah
                if M[i][j] != 0:
                    print(f"Iterasi ke-{step}: Tukar baris ke-{j+1} dengan baris ke-{i+1}")
                    M[j], M[i] = M[i], M[j] # menukar baris
                    tampilkanMatrix(M)
                    print()
                    step += 1
                    swap += 1
                    pivot = M[j][j]
                    break
        
        if pivot == 0: continue # jaga2 kalo masih 0 kita continue ke kolom berikutnya bro

        # normalisasi pivot menjadi 1 utama
        if pivot != 1:
            print(f"Iterasi ke-{step}: Normalisasi baris ke-{j+1} / ({pivot:.2f})")
            print()
            for c in range(col): # loop tiap kolom di baris ke j (karena pivot di M[j][j])
                M[j][c] /= pivot
            tampilkanMatrix(M)
            print()
            step += 1

        # eliminasi ke semua baris di bawahnya dulu
        for r in range(j+1, row):
            if r == j or M[r][j] == 0: 
                continue 

            pengali = M[r][j]
            print(f"Iterasi ke-{step}: Baris ke-{r+1} - ({pengali:.2f}) * Baris ke-{j+1}")
            print()
            for c in range(col):
                M[r][c] -= pengali * M[j][c]

            tampilkanMatrix(M)
            print()
            step += 1

    for j in range(row-1, -1, -1):
        pivot = M[j][j]
        # skip normalisasi karena sudah dinormalisasi langsung ke eliminasi
        for r in range(j-1, -1, -1):
            if r == j or M[r][j] == 0: 
                continue 

            pengali = M[r][j]
            print(f"Iterasi ke-{step}: Baris ke-{r+1} - ({pengali:.2f}) * Baris ke-{j+1}")
            print()
            for c in range(col):
                M[r][c] -= pengali * M[j][c]

            tampilkanMatrix(M)
            print()
            step += 1
    
    print("Matrix diagonal akhir:")
    print()
    tampilkanMatrix(M)
    print()
    print("Atau persamaan:")
    tampilkanPersamaan(M)
    return M

def ambilSolusi(matrix):
    col = len(matrix[0])
    variabel = col - 1

    solusi = []

    for i in range(variabel):
        if abs(matrix[i][i]) < 1e-9: # cek apakah diagonal mendekati 0
            if abs(matrix[i][col-1]) < 1e-9: # cek apakah konstanta mendekati 0
                return None, "Gak terhingga"
            else:
                return None, "Gak onk"
        
        solusi.append(matrix[i][col-1])

    return solusi, "wes onk, aman"

def showStepGaussJordanElimination(matrix):
    print(f"===Penyelesaian SPL menggunakan OBE Gauss-Jordan===")
    print()
    print("Input matrixnya:")
    tampilkanMatrix(matrix)
    print()
    print("Atau persamaan:")
    tampilkanPersamaan(matrix)
    print()
    row = len(matrix)
    col = len(matrix[0])

    if row < col - 1: # kondisi variabel bebas
        print(f"Jumlah persamaan tidak cukup, akan ada {col - row - 1} variabel bebas")
        return

    M = iterateGaussJordan(matrix)
    solusi, status = ambilSolusi(M)

    if status == "Gak onk":
        print("Tidak ada solusi (persamaan tidak konsisten)")
    elif status == "Gak terhingga":
        print("Solusi tak hingga (sistem memiliki variabel nol semua / ada variabel bebas)")
    else:
        print()
        print("Solusinya:")
        for i, res in enumerate(solusi):
            print(f"X{i+1} = {res:.2f}")


# main driver
inputMatrix = [
    [1, 1, 1, 1],
    [1, 2, 3, 1],
    [1, 4, 9, 1],
]

showStepGaussJordanElimination(inputMatrix)

### Program PPT2 - Invers Matrix menggunakan OBE Gauss-Jordan

Pseudocode:

1. Buat matrix input
2. Buat function calls ke function utama
3. Di function utama, buat utility yang diinginkan misal print judul / header, matrix input, dan sebagainya
4. Cek dulu matrixnya kalau banyak baris tidak sama dengan banyak kolom maka tampilkan pesan error (di function helper)
5. Buat matrix identity.
5. Di function utama bisa langsung buat algoritma gauss jordan:
    - Ambil pivot di diagonal yang bukan 0, jika 0 tukar baris dengan baris di bawahnya
    - Normalisasi pivot agar menjadi 1 utama (jika sudah satu utama skip)
    - Lakukan pengurangan untuk baris di bawahnya agar di bawah 1 utama semuanya 0
    - Lakukan terus hingga kolom terakhir sudah tidak ada baris di bawah 1 utama, maka lakukan lagi tetapi untuk baris di atasnya dengan algoritma yang sama
    - Hal yang sama dilakukan di matrix identity (lakukan bersamaan dengan input matrix)
    - Tampilkan hasil matrix augmented setiap iterasi
6. Tampilkan output hasil invers matrix
done!

In [ ]:
def usingGaussJordanInversion(M, I):
    row = len(M)
    col = len(M[0])
    step = 1
    swap = 0

    print("Proses OBE")
    print()

    for j in range(row):
        pivot = M[j][j]

        if pivot == 0:
            for i in range(j+1, row):
                if M[i][j] != 0:
                    print(f"Iterasi ke-{step}: Tukar baris ke-{j+1} dengan baris ke-{i+1}")
                    M[i], M[j] = M[j], M[i]
                    I[i], I[j] = I[j], I[i]
                    print("Matrix menjadi: ")
                    aug = []
                    for baris in range(row):
                        aug.append(M[baris] + I[baris])

                    tampilkanMatrix(aug)
                    print()
            
        if pivot == 0: continue # jaga2 kalo masih 0 kita continue ke kolom berikutnya bro

        # normalisasi pivot menjadi 1 utama
        if pivot != 1:
            print(f"Iterasi ke-{step}: Normalisasi baris ke-{j+1} / ({pivot:.2f})")
            print()
            for c in range(row): # loop tiap kolom di baris ke j (karena pivot di M[j][j])
                M[j][c] /= pivot
                I[j][c] /= pivot

            aug = []
            for baris in range(row):
                aug.append(M[baris] + I[baris])

            tampilkanMatrix(aug)
            print()
            step += 1

        # eliminasi ke semua baris di bawahnya dulu
        for r in range(j+1, row):
            if r == j or M[r][j] == 0: 
                continue 

            pengali = M[r][j]
            print(f"Iterasi ke-{step}: Baris ke-{r+1} - ({pengali:.2f}) * Baris ke-{j+1}")
            print()
            for c in range(col):
                M[r][c] -= pengali * M[j][c]
                I[r][c] -= pengali * I[j][c]

            aug = []
            for baris in range(row):
                aug.append(M[baris] + I[baris])

            tampilkanMatrix(aug)
            print()
            step += 1

    for j in range(row-1, -1, -1):
        pivot = M[j][j]
        # skip normalisasi karena sudah dinormalisasi langsung ke eliminasi
        for r in range(j-1, -1, -1):
            if r == j or M[r][j] == 0: 
                continue 

            pengali = M[r][j]
            print(f"Iterasi ke-{step}: Baris ke-{r+1} - ({pengali:.2f}) * Baris ke-{j+1}")
            print()
            for c in range(col):
                M[r][c] -= pengali * M[j][c]
                I[r][c] -= pengali * I[j][c]

            aug = []
            for baris in range(row):
                aug.append(M[baris] + I[baris])

            tampilkanMatrix(aug)
            print()
            step += 1
    
    return M, I, swap




def showStepGaussJordanInversion(matrix):
    print(f"===Invers Matrix Menggunakan Gauss Jordan")
    print()
    print("Input matrixnya:")
    tampilkanMatrix(matrix)
    print()

    row = len(matrix)
    col = len(matrix[0])

    M = [r[:] for r in matrix]
    I = [] # buat matrix identitas

    if row != col: 
        print("Tidak dapat ditentukan inversnya")
        invalidInputMessageDeterminan(row, col)
        return
    else:
        for i in range(row):
            I.append([])
            for j in range(col):
                if j == i:
                    I[i].append(1)
                else:
                    I[i].append(0)
    
    M , I, swap = usingGaussJordanInversion(M, I)

    print("Matrix awal:")
    tampilkanMatrix(matrix)
    print()
    print("Menjadi identitas:")
    tampilkanMatrix(M)
    print()
    print("Matrix inversnya:")
    tampilkanMatrix(I)
    print()
    print(f"Telah dilakukan {swap} kali tukar baris")


# main driver
inputMatrix = [
    [1, 1, 1],
    [1, 2, 3],
    [1, 4, 9],
]

showStepGaussJordanInversion(inputMatrix)

### Program PPT3 - Determinan Matrix Menggunakan OBE Gauss

Pseudocode:

1. Buat matrix input
2. Buat function calls ke function utama
3. Di function utama, buat utility yang diinginkan misal print judul / header, matrix input, dan sebagainya
4. Cek dulu matrixnya kalau banyak baris tidak sama dengan banyak kolom maka tampilkan pesan error (di function helper)
5. Di function utama bisa langsung buat algoritma gauss jordan:
    - Ambil pivot di diagonal yang bukan 0, jika 0 tukar baris dengan baris di bawahnya
    - Normalisasi pivot agar menjadi 1 utama (jika sudah satu utama skip)
    - Lakukan pengurangan untuk baris di bawahnya agar di bawah 1 utama semuanya 0
    - Lakukan terus hingga kolom terakhir sudah tidak ada baris di bawah 1 utama, maka lakukan lagi tetapi untuk baris di atasnya dengan algoritma yang sama
6. Tampilkan output hasil determinan.
done!

In [ ]:
def gaussAlgorithm(matrix):
    swap = 0        # hitung berapa kali tukar baris
    row = len(matrix)
    M = [r[:] for r in matrix]      # copy matrix ke M, aku gk mau modif matrix awal, resiko kehilangan data awal
    step = 1

    for j in range(row): # loop tiap kolom, jadi dia bisa drive gauss (sebagai index baris) sekaligus nentuin pivot
        pivot = M[j][j]
        if pivot == 0:
            # lakukan tukar baris
            for i in range(j+1, row): # loop tiap baris, cari pivot != 0 di bagian bawah
                if M[i][j] != 0:
                    print(f"Iterasi ke-{step}: Tukar baris ke-{j+1} dengan baris ke-{i+1}")
                    print()
                    M[j], M[i] = M[i], M[j] # menukar baris
                    tampilkanMatrix(M)
                    print()
                    step += 1
                    swap += 1
                    pivot = M[j][j]
                    break
        
        # sudah ketemu pivot kita eliminasi
        for i in range(j+1, row): # loop tiap baris ke bawah, kek di bagian tukar baris
            if M[i][j] == 0: continue

            pengali = M[i][j] / pivot
            print(f"Iterasi ke-{step}: Baris ke-{i+1} = Baris ke-{i+1} - ({pengali:.2f}) * Baris ke-{j+1}")
            print()

            for c in range(row): # loop tiap kolom buat OBE. Kita pakai row juga soalnya row == col (bisa dihitung determinannya)
                M[i][c] -= pengali * M[j][c] # j adalah index baris si pivot. Karena pivotnya selalu M[j][j]

            tampilkanMatrix(M)
            print()
            step += 1

    return M, swap


def showStepGaussDeterminant(matrix):
    print(f"===Determinan matrix dengan Algoritma Gauss===")
    print()
    print("Matrix Input:")

    tampilkanMatrix(matrix)
    print()
    row = len(matrix)
    col = len(matrix[0])

    if row == col:
        print(f"Iterasi OBE gauss untuk menentukan determinan matrix {row} x {col}")
        print()
        M, swap = gaussAlgorithm(matrix)
        det = 1
        elemenDiagonal = []

        for i in range(row):
            elemenDiagonal.append(f"({M[i][i]:.2f})")
            det *= M[i][i]

        tandaSwap = pow(-1, swap)
        det *= tandaSwap
        print("Hasil determinan matrix input:")
        tampilkanMatrix(matrix)
        print()

        print("Menjadi matrix segitiga atas:")
        tampilkanMatrix(M)
        print()

        print("Determinan = " + " * ".join(elemenDiagonal))
        print(f"Hasil akhir: {det:.2f}, karena ada {swap} kali tukar baris")
        print()

    else:
        invalidInputMessageDeterminan(row, col)
                    
                    
# main driver
inputMatrix = [
    [2, 1, -1, 8],
    [-3, -1, 2, -11],
    [-2, 1, 2, -3],
    [2, 4, 1, 3]
]

showStepGaussDeterminant(inputMatrix)

Program PPT4-Determinan dengan Ekspansi Kofaktor

Pseudocode:

1. Buat matrix input
2. Buat function calls ke function utama
3. Di function utama, buat utility yang diinginkan misal print judul / header, matrix input, dan sebagainya
4. Cek dulu matrixnya kalau banyak baris tidak sama dengan banyak kolom maka tampilkan pesan error (di function helper)
5. Di function utama bisa langsung buat algoritma ekspansi kofaktor (disini menggunakan baris pertama):
    - ambil elemen baris 1 kolom 1
    - Tentukan minor dan determinannya (menggunakan numpy)
    - kalikan elemen dengan tanda pada kofaktor dan dikalikan dengan determinan kofaktor
    - tambahkan hasil tadi ke determinan total
    - ulangi sampai baris 1 kolom terakhir
6. Tampilkan output hasil determinannya
done!

In [ ]:
def minorMatrix(matrix, row, col):
    minor = []

    for i in range(len(matrix)):
        if i == row:
            continue

        newBaris = []

        for j in range(len(matrix[i])):
            if j == col:
                continue

            newBaris.append(matrix[i][j])
        
        minor.append(newBaris)

    return minor

def showStep(matrix):
    print("=== Determinan Menggunakan Ekspansi Kofaktor Baris kesatu ===")
    print("Matrix input:")
    tampilkanMatrix(matrix)
    print()
    row = len(matrix)
    col = len(matrix[0])

    if row == col:
        print(f"Determinan dari matrix input ukuran {row} x {col}:")
        det = 0
        kalkulasi = []
        for j in range (row):
            tanda = pow(-1, j)
            elemen = matrix[0][j]
            print(f"Elemen ke-{j+1}: {elemen:.2f}")
            if tanda > 0: 
                print("Tanda: Positif")
            else:
                print("Tanda: Negatif")
            
            print("Minornya:")
            minor = minorMatrix(matrix, 0, j)
            tampilkanMatrix(minor)
            print(f"Determinan minornya: {hitungDet(minor):.2f}")
            print()

            kalkulasi.append(f"{tanda:.2f} * {elemen:.2f} * det(Minor[0][{j}])")
            det += tanda * elemen * hitungDet(minor)

        print("Hasil perhitungan lengkap determinannya:")
        print("Determinan = " + " + ".join(kalkulasi))
        print()
        print(f"Hasil akhir determinan = {det:.2f}")
        print()
    else:
        invalidInputMessageDeterminan(row, col)


# main driver
inputMatrix = [
    [2, 1, -1, 8],
    [-3, -1, 2, -11],
    [-2, 1, 2, -3],
    [2, 4, 1, 3]
]

showStep(inputMatrix)




### Program PPT5-Operasi Vektor Pada Ruang Eauclid (Penambahan, Pengurangan, Norma, Jarak, dsb.)

Pseudocode:
1. Buat 5 pilihan menggunakan variabel yang memuat opsi dan branch switch (menggunakan match juga bisa)
2. Buat 5 function yaitu fungsi penambahan vektor, pengurangan, norma, jarak, dan perkalian skalar. Buat semua fungsi agar dapat memuat banyak vektor sekaligus.
3. Buat masing-masing algoritma function sesuai di PPT-5: [disini](docs/PPT5.pdf)

In [ ]:
def tambah(vectors):
    print("\n=== Kamu memilih operasi tambah vektor ===\n")

    print("Vector inputnya:\n")

    displayInput(vectors)

    # cari panjang maksimum vector
    # maxLen = 0

    # for i, v in enumerate(vectors):
    #     if len(v) > maxLen:
    #         maxLen = len(v)

    maxLen = max(len(v) for v in vectors)
    
    # penjumlahan semua vector
    W = [0] * maxLen # set semua elemen W ke 0
    for v in vectors:
        for i in range(len(v)):
            W[i] += v[i] # tambah tiap ada vector ke result

    print("=== Hasil penjumlahan vektor ===\n")
    print("    W =", end=' ')
    tampilkanMatrix1D(W)
    print()

    # Warning message jika ukuran berbeda
    if len(set(len(v) for v in vectors)) > 1:
        print(f"Perhatian: ukuran vektor berbeda, elemen yang tidak ada dianggap 0.\n")

def kurangi(vectors):
    # cari panjang maksimum vector
    maxLen = 0
    maxIdx = 0

    for i, v in enumerate(vectors):
        if len(v) > maxLen:
            maxLen = len(v)
            maxIdx = i
    
    W = list(vectors[maxIdx])

    print("\n=== Kamu memilih operasi pengurangan vektor ===\n")
    print("Perhatian: sistem memilih vektor terpanjang yang akan dikurangi vektor yang lebih pendek (atau sesuai urutan input)\n")
    print("Vektor terpanjang adalah W\n")
    print("    W =", end = ' ')
    tampilkanMatrix1D(W)
    print()

    print("Akan dikurangi dengan:\n")
    for i, v in enumerate(vectors):
        if i == maxIdx: continue
        print(f"    V{i+1} =", end=' ')
        tampilkanMatrix1D(v)
        print()

    # operasi pengurangan
    for v in vectors:
        if len(v) == maxLen: continue
        for i in range(len(v)):
            W[i] -= v[i] # kurangi tiap ada vector ke result

    print("=== Hasil pengurangan vektor ===\n")
    print("    W =", end=' ')
    tampilkanMatrix1D(W)
    print()

    # Warning message jika ukuran berbeda
    if len(set(len(v) for v in vectors)) > 1:
        print(f"Perhatian: ukuran vektor berbeda, elemen yang tidak ada dianggap 0.\n")

def kaliSkalar(vectors, skalar):
    print("\n=== Kamu memilih operasi perkalian skalar dengan vektor ===\n")
    print("Vector inputnya:\n")

    displayInput(vectors)

    print(f"Input skalarnya: {skalar}\n")

    for v in vectors:
        for i in range(len(v)):
            v[i] *= skalar

    print("=== Hasil pengurangan vektor ===\n")

    # tampilkan semua matrix
    for i, v in enumerate(vectors):
        print(f"    V{i+1} =", end=' ')
        tampilkanMatrix1D(v)
        print()

def normaVector(vectors):
    print("\n=== Kamu memilih operasi norma vector ===\n")
    print("Vector inputnya:\n")

    displayInput(vectors)

    norm = [0] * len(vectors)
    kalkulasi = []
    for i, v in enumerate(vectors):
        kalkulasi.append([])
        for j in range(len(v)):
            norm[i] += pow(v[j], 2)
            kalkulasi[i].append(f"({v[j]})^2")

    norm = np.sqrt(norm)

    print("=== Hasil norma vektor ===\n")
    for i in range(len(vectors)):
        print(f"    Norma V{i+1} = sqrt[" + " + ".join(kalkulasi[i]) + "]")
        print(f"    Hasil akhir = {norm[i]:.2f}")
        print()

    print("FYI: sqrt(x) = akar kuadrat dari x atau x^0.5")

def jarakVector(vectors):
    print("\n=== Kamu memilih operasi jarak vector ===\n")
    print("Vector inputnya:\n")

    displayInput(vectors)

    print("=== Hasil jarak kedua vektor masing-masing ===\n")
    for i, Uvec in enumerate(vectors):
        for j, Vvec in enumerate(vectors[i+1:], start = i+1):
            # alternatif pakai itertools ( from itertools import combinations )
            # for (i, U), (j, V) in combinations(enumerate(vectors), 2):
            U = Uvec
            V = Vvec

            maxLen = max(len(U), len(V))
            kalkulasi = []
            jarak = 0.0

            for n in range(maxLen):
                Un = U[n] if n < len(U) else 0
                Vn = V[n] if n < len(V) else 0
                jarak += pow(Un - Vn, 2)
                kalkulasi.append(f"({Un:.2f} - {Vn:.2f})^2")

            jarak = np.sqrt(jarak)
            print(f"    Jarak V{i+1} ke V{j+1} = sqrt[" + " + ".join(kalkulasi) + "]")
            print(f"    Jarak akhir = {jarak:.2f}\n")

            # Warning message jika ukuran berbeda
            if len(U) != len(V):
                print(f"    Perhatian: ukuran vektor berbeda, elemen yang tidak ada dianggap 0.\n")

# main driver-------------------------
# pilih operasi
opsi = 5
# vector input
u = [1, 2, 3]
v = [4, 5]
j = [3, 4, 5]
k = [1, 3, 0.2]
l = [0.0, 7, 6.6352, 932, 329839]

vectors = [u, v, j, k, l]

# kalo perkalian skalar
skalar = 2

# bracnch ke operasi yang dipilih
match opsi:
    case 1:
        tambah(vectors)
    case 2:
        kurangi(vectors)
    case 3: 
        kaliSkalar(vectors, skalar)
    case 4:
        normaVector(vectors)
    case 5:
        jarakVector(vectors)

### Program PPT6 - Operasi Dot Product, Proyeksi Orthogonal, dsb.

pseudocode:
1. Buat 4 pilihan case menggunakan match atau switch
2. Buat 4 macam input untuk testing keempat pilihan case
3. Buat fungsi dan algoritmanya sesuai materi PPT6: [disini](docs/PPT6.pdf)
4. Kita mungkin membutuhkan helper function tambahan dan memang sudah ditambahkan ke helper function general dan helper function spesific

In [ ]:
def panjangVector(V):
    res = 0
    for i in V:
        a = i * i
        res += a

    res = np.sqrt(res)
    return res

def radToDeg(res):
    ans = res * (180 / np.pi)
    return ans

def quickDot(u, v):
    res = np.dot(u, v)
    return res

def proy(U, V):
    resDot = quickDot(U, V)
    lengU = panjangVector(U)

    W1 = quickDot((resDot / pow(lengU, 2)), U)
    W2 = []

    for k, vec in enumerate(W1):
        W2.append(V[k] - vec)

    print(f"    Hasil W1 =", end = ' ')
    tampilkanMatrix1D(W1)
    print(f"    Hasil W2 =", end = ' ')
    tampilkanMatrix1D(W2)
    print()

def dotProduct(vectors):
    print("\n=== Kamu memilih operasi dot product===\n")
    print("Vektor input:\n")

    displayInput(vectors)

    print("=== Hasil dot product & sudutnya ===\n")
    for i, Uvec in enumerate(vectors):
        for j, Vvec in enumerate(vectors[i+1:], start = i+1):
            U = Uvec
            V = Vvec
            lengU = len(U)
            lengV = len(V)
            if lengU != lengV:
                print(f"Dot product tidak dapat dilakukan karena vector berbeda dimensi (V{i+1}: {lengU}d ; V{j+1}: {lengV}d)\n")
            else:
                kalkulasi = []
                ans = 0.0

                for n in range(lengV):
                    Un = U[n]
                    Vn = V[n]
                    ans += Un * Vn
                    kalkulasi.append(f"({Un:.2f} * {Vn:.2f})")

                print(f"    Hasil V{i+1} dot V{j+1} = [" + " + ".join(kalkulasi) + "]")
                print(f"    Hasil akhir = {ans:.2f}\n")

                # hitung sudut antara vector
                pjgU = panjangVector(U)
                pjgV = panjangVector(V)
                resRad = 0.0

                resRad = np.acos(ans / (pjgU * pjgV))
                resDeg = radToDeg(resRad)
                print(f"    Hasil hitung sudut V{i+1} dengan V{j+1} = arc cos({ans:.2f} / ({pjgU:.2f}) * ({pjgV:.2f}))")
                print(f"    Hasil sudut (rad) = {resRad:.2f} rad")
                print(f"    Hasil sudut (Deg): {resDeg:.2f}\n")

def proyeksiOrthogonal(vectors):
    print("\n=== Kamu memilih operasi proyeksi orthogonal ===\n")
    print("Vektor input:\n")

    displayInput(vectors)

    print("=== Hasil proyeksi orthogonal ===\n")
    for i, Uvec in enumerate(vectors):
        for j, Vvec in enumerate(vectors[i+1:], start = i+1):
            U = Uvec
            V = Vvec
            print(f"Hasil proyeksi orthogonal V{j+1} pada vector V{i+1}:")
            proy(U, V)

            print("Sebaliknya:")
            print(f"Hasil proyeksi orthogonal V{i+1} pada vector V{j+1}:")
            proy(V, U)

def hitungJrkTtkKeGrs2d(points, equations):
    print("\n=== Kamu memilih operasi hitung jarak titik ke garis===\n")
    print("Persamaan garis input:\n")

    displayInput2(equations)
    print(f"Titik uji:\n")
    displayInput3(points)

    print("=== Hasil hitung jarak titik ke garis ===\n")
    for i, vpoint in enumerate(points):
        for j, vequation in enumerate(equations):
            pt = vpoint
            ptx = pt[0]
            pty = pt[1]

            eq = vequation
            ans = 0.0

            if len(eq) < 2:
                print(f"Persamaan E{j+1} tidak memiliki komponen C / konstanta, tidak dapat dihitung\n")
                return

            A = eq[0]
            B = eq[1] if len(eq) > 2 else 0.0
            C = eq[2] if len(eq) > 2 else eq[1]
            # pembilang = abs((ptx * eq[0]) + 
            #                 (pty * eq[1] if len(eq) > 2 else 0) +
            #                 (eq[2] if len(eq) > 2 else eq[1]))

            pembilang = abs((ptx * A) + (pty * B) + C)
            # penyebut = np.sqrt((pow(eq[0], 2)) + (pow(eq[1], 2) if len(eq) > 2 else 0))
            penyebut = np.sqrt(pow(A, 2) + pow(B, 2))

            ans = pembilang / penyebut

            # print(f"Jarak titik P{i+1} ke garis E{j+1} = abs({eq[0]:.2f} * {ptx:.2f} + {eq[1]:.2f if len(eq) > 2 else 0.00} * {pty:.2f} + {eq[2]:.2f if len(eq) > 2 else eq[1]:.2f})") ... this is error due to :.2f is not in expression

            print(f"Jarak titik P{i+1} ke garis E{j+1} = abs(({A:.2f} * {ptx:.2f}) + ({B:.2f} * {pty:.2f}) + ({C:.2f})) / sqrt(({A:.2f})^2 + ({B:.2f})^2)")

            print(f"Hasil jarak akhir = {ans:.2f}\n")

def dotProductRT(sum, sub):
    print("\n=== Kamu memilih operasi dot product r+t , r-t ===\n")

    print("r+t input:\n")
    displayInput4(sum)

    print("r-t input:\n")
    displayInput5(sub)

    if len(sum) != len(sub):
        print(f"Dapat dilihat bahwa banyak elemen tidak sama dimana len(r+t) = {len(sum)} sedangkan len(r-t) = {len(sub)}, tidak dapat dihitung r.t")

    print("=== Hasil hitung r dot t ===\n")
    for i in range(len(sum)):
        plus = sum[i]
        minus = sub[i]

        if len(plus) != len(minus):
            print(f"Dapat dilihat bahwa banyak elemen tidak sama dimana len(r+t) = {len(plus)} sedangkan len(r-t) = {len(minus)}, tidak dapat dihitung r.t")

        sqrPlus = pow(panjangVector(plus), 2) / 4
        sqrMin = pow(panjangVector(minus), 2) / 4

        ans = sqrPlus - sqrMin

        print(f"    Hasil r dot t [{i+1}] = 0.25(|{sqrPlus:.2f}|^2 - |{sqrMin:.2f}|^2)")
        print(f"    Hasil akhirnya = {ans:.2f}\n")



# main driver
opsi = 1

U = [1, 2, 3]
V = [4, 5, 6]
J = [7, 8, 9]
K = [10, 11, 12]
L = [13, 14, 15]

# D = [-3, 5, 12]
# E = [5, 7, -2]
# vectors = [D, E]

vectors = [U, V, J, K, L]

point1 = [1, 1]
point2 = [-5, 8]
E1 = [5, -4, 9]
E2 = [3, 4, -10]
equations = [E1, E2]
points = [point1, point2]

sm1 = [2, 3, 4]
sm2 = [4, 5, 6]
sm3 = [7, 8, 9]
sb1 = [9, 8, 7]
sb2 = [7, 5, 6]
sb3 = [5, 4, 2]

sum = [sm1, sm2, sm3]
sub = [sb1, sb2, sb3]

match opsi:
    case 1:
        dotProduct(vectors)
    case 2:
        proyeksiOrthogonal(vectors)
    case 3:
        hitungJrkTtkKeGrs2d(points, equations)
    case 4:
        dotProductRT(sum, sub)

### Program PPT7 - Operasi Cross Product, Hitung Luas Segitiga, etc.

Pseudocode:
1. Buat opsi seperti sebelumnya
2. Buat function sebanyak opsi. Acuan: [PPT7](docs/PPT7.pdf)
3. Sedikit modifikasi menjadi main() agar error message ditampilkan sebelum eksekusi function

In [ ]:
def displayInput3(vectors):
    # tampilkan semua matrix
    for i, v in enumerate(vectors):
        print(f"    P{i+1} =", end=' ')
        tampilkanMatrix1D(v)
        print()

def displayInput8(vectors):
    # tampilkan semua matrix
    for i, v in enumerate(vectors):
        print(f"    T{i+1} =", end = ' ')
        tampilkanMatrix1D(v)
        print()

def crossProductDeterminant(Uvectors, Vvectors):
    print("\n=== Kamu memilih operasi cross product ===\n")
    print("Vektor input:\n")

    displayInput6(Uvectors)
    displayInput(Vvectors)

    # error message kalo ukuran tidak 3
    # if len(Uvectors) != 3 or len(Vvectors):
    #     print(f"Tidak dapat menghitung cross product karena salah satu ukuran tidak 3\n")
    #     return

    print("=== Hasil cross product menggunakan determinan ===\n")
    for i, Uvec in enumerate(Uvectors):
        for j, Vvec in enumerate(Vvectors):
            U = Uvec
            V = Vvec

            if len(U) != len(V) or len(U) > 3 or len(V) > 3:
                print(f"Panjang dari vektor U berbeda dengan vektor V. Len(U) = {len(U)} sedangkan len(V) = {len(V)}")
                print("U x V terdefinisi hanya pada vektor dimensi 3 saja")

            W1 = []
            for k in range(2):
                W1.append([])
                AppendA = U[1] if k == 0 else V[1]
                AppendB = U[2] if k == 0 else V[2]
                W1[k].append(AppendA)
                W1[k].append(AppendB)

            W2 = []
            for k in range(2):
                W2.append([])
                AppendA = U[0] if k == 0 else V[0]
                AppendB = U[2] if k == 0 else V[2]
                W2[k].append(AppendA)
                W2[k].append(AppendB)

            W3 = []
            for k in range(2):
                W3.append([])
                AppendA = U[0] if k == 0 else V[0]
                AppendB = U[1] if k == 0 else V[1]
                W3[k].append(AppendA)
                W3[k].append(AppendB)

            W = [W1, W2, W3]

            print(f"-----------------------------------------------------------------------")
            print(f"|========================== Operasi U{i+1} X V{j+1} ==========================|")
            print(f"-----------------------------------------------------------------------\n")

            print(f"====| Rumus1 U{i+1} X V{j+1} = [det(W1), -det(W2), det(W3)] |====\n")
            print("    Vektor W yang terbentuk adalah:\n")
            displayInput7(W)

            kalkulasi = []
            kalkulasi.append([])

            # detW1 = 0
            # detW2 = 0
            # detW3 = 0
            detW = [0, 0, 0]

            for k in range(3):
                detW[k] += hitungDet(W[k])
                kalkulasi[0].append(f"[({W[k][0][0]:.2f}) * ({W[k][1][1]:.2f})] - [({W[k][1][0]:.2f}) * ({W[k][0][1]:.2f})]")

            detW[1] *= -1

            # print(f"    Determinan W1 = " + " + ".join(kalkulasi[0][0]))
            # print(f"    Determinan W1 = {detW[0]}")
            # print(f"    Determinan W1 = " + " + ".join(kalkulasi[0][1]))
            # print(f"    Determinan W2 = {detW[1]}")
            # print(f"    Determinan W1 = " + " + ".join(kalkulasi[0][2]))
            # print(f"    Determinan W3 = {detW[2]}\n")

            for k, detK in enumerate(detW):
                print(f"    Determinan W{k+1} = {kalkulasi[0][k]}")
                print(f"    Determinan W{k+1} = {detK:.2f}")
                print()

            print(f"    Sehingga hasil U{i+1} X V{j+1} =", end = ' ')
            tampilkanMatrix1D(detW)
            print()
            print(f"==== Rumus2 ||U{i+1} X V{j+1}|| = sqrt(W1^2 + W2^2 + W3^2) ====\n")

            kalkulasi.append([])
            ans = 0
            for k, w in enumerate(detW):
                kalkulasi[0].append(f"({w:.2f})^2")
                ans += pow(w, 2)
            
            ans = np.sqrt(ans)

            print(f"    Hasil ||U{i+1} X V{j+1}|| = sqrt[" + " + ".join(kalkulasi[1]) + "]")
            print(f"    Hasil ||U{i+1} X V{j+1}|| = {ans:.2f}\n")

            print(f"==== Rumus3 menentukan theta dari ||U{i+1} X V{j+1}|| = ||U|| dot ||V|| * sin(theta) ====\n")

            res = 0.00
            lengU = 0.00
            lengV = 0.00
            
            for k in U:
                lengU += pow(k, 2)

            lengU = np.sqrt(lengU)

            for k in V:
                lengV += pow(k, 2)

            lengV = np.sqrt(lengV)

            resRad = np.asin(ans / (lengU * lengV))

            resDeg = radToDeg(resRad)

            kalkulasi.append([])
            kalkulasi[1].append(f"({lengU:.2f}) * ({lengV:.2f}) * sin(theta)")
            kalkulasi[1].append(f"asin[{ans:.2f} / (({lengU:.2f}) * ({lengV:.2f}))]")

            print(f"    Hasil ||U{i+1} X V{j+1}|| = {kalkulasi[1][0]}")
            print(f"    Hasil theta = {kalkulasi[1][1]}\n")
            print(f"    Theta = {resRad:.2f} Rad")
            print(f"    Theta = {resDeg:.2f} Deg\n")

def hitungLSegitiga(vectorsP):
    print("\n=== Kamu memilih operasi hitung luas segitiga Product ===\n")
    print("Vektor input:\n")

    displayInput3(vectorsP)
    
    leng = len(vectorsP)
    for i in range(leng):
        pusat = vectorsP[i]

        print(f"=== Luas segitiga dengan pusat P{i+1} ===\n")

        P1 = vectorsP[(i+1) % leng]
        P2 = vectorsP[(i+2) % leng]

        PP1 = np.array(P1) - np.array(pusat)
        PP2 = np.array(P2) - np.array(pusat)
        
        crossP = np.cross(PP1, PP2)
        lengPP = np.linalg.norm(crossP)

        lengPP = np.sqrt(lengPP)

        luas = 0.5 * (lengPP)

        idxP1 = (i+1) % leng
        idxP2 = (i+2) % leng
        print(f"    Rumus luas segitiga = 0.5 * ||P{i+1}P{idxP1+1} X P{i+1}P{idxP2+1}||")
        print(f"    Hasil luas segitiga = {luas:.2f}\n")

def persBdgMll1Ttk(titik, nVal):
    print("\n=== Kamu memilih operasi hitung persamaan bidang melalui 1 titik ===\n")
    print("Vektor input:\n")

    print(f"    Titik =", end = ' ')
    tampilkanMatrix1D(titik)
    print(f"    Tegak lurus terhadap n =", end = ' ')
    tampilkanMatrix1D(nVal)
    print()

    constanta = 0.00
    for i in range(3):
        constanta += ((titik[i] * -1) * nVal[i])

    print(f"Rumus: P0P * n = 0")
    print(f"Persamaannya: ({nVal[0]:.2f})X + ({nVal[1]:.2f})Y + ({nVal[2]:.2f})Z + ({constanta:.2f}) = 0\n")

def persBdgMll3Ttk(Tvec):
    print("\n=== Kamu memilih operasi hitung persamaan bidang melalui 3 titik ===\n")
    print("Vektor input:\n")

    displayInput8(Tvec)

    leng = len(Tvec)
    for i, pusat in enumerate(Tvec):
        P1 = Tvec[(i+1) % leng]
        P2 = Tvec[(i+2) % leng]

        PP1 = np.array(P1) - np.array(pusat)
        PP2 = np.array(P2) - np.array(pusat)

        crossPP = np.cross(PP1, PP2)

        constanta = 0.00
        for j in range(3):
            constanta += ((pusat[j] * -1) * crossPP[j])

        print(f"Persamaan bidang dengan P0 = P{i+j}:\n")
        print(f"    Rumus: P0P * n")
        print(f"    Persamaan: ({crossPP[0]:.2f})X + ({crossPP[1]:.2f})Y + ({crossPP[2]:.2f})Z + ({constanta:.2f}) = 0\n")


# main driver 
def main():
    opsi = 4

    U1 = [1, 2, 3]
    U2 = [73, 63, 9]
    U3 = [-7, -9, 10]

    V1 = [4, 5, 6]
    V2 = [2, -7, 9]
    V3 = [102, -87, -90]

    vectorsU = [U1, U2, U3]
    vectorsV = [V1, V2, V3]

    P1 = [9, 201, -29]
    P2 = [2, -8, -13]
    P3 = [1, 2, 3]

    vectorsP = [P1, P2, P3]

    titik = [3, -1, 7]
    nVal = [4, 2, -5]

    T1 = [-3, 2, 0]
    T2 = [0, -1, 2]
    T3 = [5, 1, 3]

    Tvec = [T1, T2, T3]

    match opsi:
        case 1:
            # exit kalo ukuran tidak sama dengan 3
            if len(Uvectors) != 3 or len(Vvectors):
                print(f"Tidak dapat menghitung cross product karena salah satu ukuran tidak 3\n")
                return

            crossProductDeterminant(vectorsU, vectorsV)
        case 2:
            # exit kalo ukuran tidak sama dengan 3
            if len(vectorsP[0]) != 3 or len(vectorsP[1]) != 3 or len(vectorsP[2]) != 3:
                print(f"Tidak dapat menghitung luas segitiga karena salah satu ukuran tidak 3\n")
                return
            
            hitungLSegitiga(vectorsP)
        case 3:
            # exit kalo ukuran tidak sama dengan 3
            if len(titik) != 3 or len(nVal) != 3:
                print(f"Tidak dapat menghitung persamaan karena salah satu ukuran tidak 3\n")
                return
            
            persBdgMll1Ttk(titik, nVal)

        case 4:
            # exit kalo ukuran tidak sama dengan 3
            if len(Tvec[0]) != 3 or len(Tvec[1]) != 3 or len(Tvec[2]) != 3 or len(Tvec) != 3:
                print(f"Tidak dapat menghitung persamaan karena salah satu ukuran tidak 3\n")
                return
            
            persBdgMll3Ttk(Tvec)

"""
int main(int argv, char **argv) {
    return 0;
}
"""

if __name__ == "__main__":
    main()

### Program PPT9 - Persamaan Parametrik, Simetrik, Transformasi, dsb.

Pseudocode:
1. Buat opsi seperti sebelumnya
2. Buat function sebanyak opsi. Acuan: [PPT9](docs/PPT9.pdf)
3. Sedikit modifikasi ```if __name__ == "__main__":``` agar sesuai standar
4. Yang aku skip adalah jarak 2 bidang sejajar sama komposisi transformasi

In [ ]:
def tampilkanPersamaan(matrix):
    row = len(matrix)
    col = len(matrix[0])
    variabel = col - 1
    vars = "XYZ"
    
    for i in range(row):
        left = []
        for j in range(variabel):
            coef = matrix[i][j]
            left.append(f"({coef:.2f}){vars[i]}")

        lhs = " + ".join(left)
        rhs = matrix[i][col-1]
        rhs *= -1

        print(f"    {lhs} = {rhs:.2f}")

def tampilkanPersamaan1D(matrix):
    variabel = len(matrix) - 1
    vars = "XYZ"

    left = []
    for j in range(variabel):
        coef = matrix[j]                     
        left.append(f"({coef:.2f}){vars[j]}")

    lhs = " + ".join(left)
    rhs = -matrix[-1]                        

    print(f"    {lhs} = {rhs:.2f}")

        

def persParam(P):
    print("\n=== Kamu memilih operasi hitung persamaan parametrik melalui 2 titik ===\n")
    print("Vektor input:\n")

    displayInput3(P)
    
    for i, P1 in enumerate(P):
        for j, P2 in enumerate(P):
            if j == i: continue

            print(f"Persamaan parametrik untuk garis l melalui titik P{i+1} dan P{j+1} dengan P0 di P{i+1}\n")
            print(f"    Rumus: P0P * V")
            PP = np.array(P1) - np.array(P2)
            print(f"    V = P2 - P1 =", end = ' ')
            tampilkanMatrix1D(PP)
            print()

            res = []
            vars = "XYZ"
            for k in range(3):
                res.append(f"{vars[k]} = {P1[k]:.2f} + ({PP[k]:.2f})t")

            print(f"    Hasil persamaan: " + "\n                     ".join(res))
            print()

def iterateGaussJordan(matrix):
    swap = 0        # hitung berapa kali tukar baris
    row = len(matrix)
    col = len(matrix[0])
    M = [r[:] for r in matrix]      # copy matrix ke M, aku gk mau modif matrix awal, resiko kehilangan data awal
    step = 1

    for j in range(row): # loop tiap kolom, jadi dia bisa drive gauss (sebagai index baris) sekaligus nentuin pivot. Perhatikan bahwa loop ini berhenti sampai row-1 karena kolom paling kanan tidak di-normalisasi
        pivot = M[j][j]
        if pivot == 0:
            # lakukan tukar baris
            for i in range(j+1, row): # loop tiap baris, cari pivot != 0 di bagian bawah
                if M[i][j] != 0:
                    print(f"    Iterasi ke-{step}: Tukar baris ke-{j+1} dengan baris ke-{i+1}")
                    M[j], M[i] = M[i], M[j] # menukar baris
                    tampilkanMatrix(M)
                    print()
                    step += 1
                    swap += 1
                    pivot = M[j][j]
                    break
        
        if pivot == 0: continue # jaga2 kalo masih 0 kita continue ke kolom berikutnya bro

        # normalisasi pivot menjadi 1 utama
        if pivot != 1:
            print(f"    Iterasi ke-{step}: Normalisasi baris ke-{j+1} / ({pivot:.2f})")
            print()
            for c in range(col): # loop tiap kolom di baris ke j (karena pivot di M[j][j])
                M[j][c] /= pivot
            tampilkanMatrix(M)
            print()
            step += 1

        # eliminasi ke semua baris di bawahnya dulu
        for r in range(j+1, row):
            if r == j or M[r][j] == 0: 
                continue 

            pengali = M[r][j]
            print(f"    Iterasi ke-{step}: Baris ke-{r+1} - ({pengali:.2f}) * Baris ke-{j+1}")
            print()
            for c in range(col):
                M[r][c] -= pengali * M[j][c]

            tampilkanMatrix(M)
            print()
            step += 1

    for j in range(row-1, -1, -1):
        pivot = M[j][j]
        # skip normalisasi karena sudah dinormalisasi langsung ke eliminasi
        for r in range(j-1, -1, -1):
            if r == j or M[r][j] == 0: 
                continue 

            pengali = M[r][j]
            print(f"    Iterasi ke-{step}: Baris ke-{r+1} - ({pengali:.2f}) * Baris ke-{j+1}")
            print()
            for c in range(col):
                M[r][c] -= pengali * M[j][c]

            tampilkanMatrix(M)
            print()
            step += 1
    
    print("    Matrix diagonal akhir:")
    print()
    tampilkanMatrix(M)
    print()
    print("    Atau persamaan:")
    tampilkanPersamaan(M)
    return M


def parametrikPotong2Bdg(E):
    print("\n=== Kamu memilih operasi hitung persamaan parametrik yang memotong 2 bidang ===\n")
    print("Input persamaan:\n")
    displayInput3(E)

    for i, E1 in enumerate(E):
        for j, E2 in enumerate(E[i+1:], start = i+1):
            ETEMP = [E1, E2]

            print(f"Persamaan yang berpotongan antara E{i+1} dengan E{j+1}:\n")
            print(f"    Rumus: X1 = X2, Y1 = Y2, Z1 = Z2\n")
            print(f"    Langkah penyelesaian dengan gauss-jordan:\n")
            SOL = iterateGaussJordan(ETEMP)
            
            res = []
            vars = "XYZ"
            for k in range(2):
                res.append(f"{vars[k]} = {SOL[k][3]:.2f} + ({-1 * SOL[k][2]:.2f})t")
            
            res.append(f"{vars[2]} = t")
            print()
            print(f"    Hasil persamaan: " + "\n                     ".join(res))

def persSimet(Simet):
    print("\n=== Kamu memilih operasi hitung persamaan simetri diketahui X, Y, Z ===\n")
    for k, S in enumerate(Simet):
        print(f"Input persamaan S{k+1}:\n")
        vars = "XYZ"
        for i in range(3):
            print(f"{vars[i]} =", end = ' ')
            tampilkanMatrix1D(S[i])
            print()

        print("Rumus: X = Y = Z\n")

        for i, S1 in enumerate(S):
            for j,S2 in enumerate(S[i+1:], start = i+1):
                print(f"    Persamaan bidang {vars[i]} dan {vars[j]} adalah:")
                koefX = S2[1]
                koefY = -1 * S1[1]
                constanta = -1 * (S1[0] * S2[1]) + S1[1] * S2[0]
                PERS = [koefX, koefY, constanta]

                tampilkanPersamaan1D(PERS)
                print()

def jrkTtkKeBdg(TITIK, BIDANG):
    print("\n=== Kamu memilih operasi hitung jarak titik ke bidang ===\n")
    print("Input titik:\n")
    displayInput8(TITIK)
    print("Input persamaan bidang:\n")
    displayInput2(BIDANG)

    print("Rumus = |ax0 + by0 + cz0 + d| / sqrt(a^2 + b^2 + c^2)\n")

    print(f"Hasil perhitungan semua kombinasi\n")
    for i, T in enumerate(TITIK):
        for j, B in enumerate(BIDANG):
            pembilang = 0.00
            abc = []
            for k in range(3):
                pembilang += (T[k] * B[k])
                abc.append(B[k])

            pembilang += B[3]

            pembilang = abs(pembilang)

            penyebut = np.linalg.norm(abc)

            res = pembilang / penyebut

            print(f"    Hasil jarak T{i+1} ke B{j+1} = {res:.2f}")

def pencerminan(Vec):
    print("\n=== Kamu memilih operasi hitung pencerminan ke sumbu dan bidang ===\n")
    print("Input vektor:\n")
    displayInput(Vec)

    for i, V in enumerate(Vec):
        print(f"Hasil pencerminan dari V{i+1}:\n")
        vars = "XYZ"
        for j in range(3):
            tmp = V[:]
            for k in range(3):
                if k == j: continue
                else:
                    tmp[k] *= -1

            print(f"    Pencerminan terhadap sumbu {vars[j]} =", end = ' ')
            tampilkanMatrix1D(tmp)

        pairs = [vars[1]+vars[2], vars[0]+vars[2], vars[0]+vars[1]]
        for j in range(2, -1, -1):
            tmp = V[:]
            tmp[j] *= -1
            print(f"    Pencerminan terhadap bidang {pairs[j]} =", end = ' ')
            tampilkanMatrix1D(tmp)

        print()

def rotasi(Vec, Deg):
    angle = degToRad(Deg)
    print("\n=== Kamu memilih operasi hitung rotasi ke sumbu ===\n")
    print("Input vektor:\n")
    displayInput(Vec)

    for i, V in enumerate(Vec):
        print(f"Hasil rotasi V{i+1} sebesar {Deg}Deg\n")
        vars = "XYZ"
        for j in range(3):
            x0, y0, z0 = V
            cs, sn = np.cos(angle), np.sin(angle)
            poros = vars[j]

            match poros:
                case 'X':
                    x = x0
                    y, z = (y0 * cs - z0 * sn), (y0 * sn + z0 * cs)
                case 'Y':
                    x, z = (x0 * cs + z0 * sn), (-x0 * sn + z0 * cs)
                    y = y0
                case 'Z':
                    x, y = (x0 * cs - y0 * sn), (x0 * sn + y0 * cs)
                    z = z0
            
            print(f"    Rotasi V{i+1} terhadap sumbu {poros} = [{x:.2f}, {y:.2f}, {z:.2f})")
        
        print()

def dilatasi(Vec, Dil):
    print("\n=== Kamu memilih operasi hitung dilatasi / kontraksi ===\n")
    print("Input vektor:\n")
    displayInput(Vec)

    for i, V in enumerate(Vec):
        for j in range(len(V)):
            V[j] *= Dil

        print(f"Hasil dilatasi V{i+1} sebesar {Dil} kali =", end = ' ')
        tampilkanMatrix1D(V)
        print()

def proyOrthogonal(Vec):
    print("\n=== Kamu memilih operasi hitung dilatasi / kontraksi ===\n")
    print("Input vektor:\n")
    displayInput(Vec)

    for i, V in enumerate(Vec):
        vars = "XYZ"
        print(f"Hasil proyeksi orthogonal V{i+1}:\n")
        for j in range(len(vars)):
            if len(V) < 3:
                for k in range(3 - len(V)):
                    V.append(0)

            x, y, z = V

            match vars[j]:
                case 'X':
                    y, z = 0, 0
                case 'Y':
                    x, z = 0, 0
                case 'Z':
                    x, y = 0, 0

            print(f"    Proyeksi orthogonal ke sumbu {vars[j]} = [{x:.2f}, {y:.2f}, {z:.2f}]")

        print()

        pairs = [vars[1]+vars[2], vars[0]+vars[2], vars[0]+vars[1]]
        for j in range(3):
            x, y, z = V

            match pairs[j]:
                case 'XY':
                    z = 0
                case 'XZ':
                    y = 0
                case 'YZ':
                    x = 0

            print(f"    Proyeksi orthogonal ke sumbu {pairs[j]} = [{x:.2f}, {y:.2f}, {z:.2f}]")
        
        print()

# main driver
if __name__ == "__main__":
    
    opsi = 8

    P1 = [2, 4, -1]
    P2 = [5, 0, 7]
    P3 = [3, -1, 0]
    P4 = [7, 0, 8]
    PARAMETRIK = [P1, P2, P3, P4]

    E1 = [-4, 5, -7, -388]
    E2 = [9, 6, -2, 215]
    EQUATION = [E1, E2]

    X1 = [3, 4]
    Y1 = [-3, 11]
    Z1 = [7, -12]
    S1 = [X1, Y1, Z1]

    SIMETRI = [S1]

    T1 = [1, -4, -3]
    T2 = [2, 7, -8]
    TITIK = [T1, T2]

    B1 = [-2, 3, -5, -7]
    B2 = [-6, 9, -15, -18]
    B3 = [2, -3, -6, -1]
    BIDANG = [B1, B2, B3]

    V1 = [2, -3, 5]
    V2 = [5, -3, 2]
    V = [V1, V2]
    SUDUTDEG = 90
    MULTIPLY = 2

    match opsi:
        case 1:
            persParam(PARAMETRIK)
        case 2:
            parametrikPotong2Bdg(EQUATION)
        case 3:
            persSimet(SIMETRI)
        case 4:
            jrkTtkKeBdg(TITIK, BIDANG)
        case 5:
            pencerminan(V)
        case 6:
            rotasi(V, SUDUTDEG)
        case 7:
            dilatasi(V, MULTIPLY)
        case 8:
            proyOrthogonal(V)   # menerima matriks ukuran 2 <= len(V) <= 3
